# SCC5977 - Aprendizado de Máquina para Séries Temporais (2024)

## Grupo
> André Guarnier De Mitri - 11395579 \
> Fabio Cavaleti - 11200550\
> Giovani Decico Lucafó - 10288779

## Problema
Incentia 11k euclidiana vs ddtw

In [ ]:
import os
import numpy as np
import torch
from typing import Tuple
from aeon.datasets import load_from_tsfile
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import check_random_state
from kan import KAN  # Ensure KAN is correctly installed and available


class KANClassifier:
    def __init__(
        self,
        width: list,
        output_dir: str = "./",
        steps: int = 20,
        k: int = 3,
        grid: int = 5,
        random_state: int = None,
    ):
        """KAN Time Series Classifier.

        Parameters
        ----------
        width : list
            The width of the KAN layers.
        output_dir : str, default = "./"
            The output directory.
        steps : int, default = 20
            The number of optimization steps.
        k : int, default = 3
            The order of piecewise polynomial.
        grid : int, default = 5
            The number of grid intervals.
        random_state : int, default = None
            The random state for the initial seed.
        """
        self.width = width
        self.output_dir = output_dir
        self.steps = steps
        self.k = k
        self.grid = grid
        self.random_state = random_state

    def _train_acc(self):
        return torch.mean(
            (
                torch.argmax(self.model(self.dataset["train_input"]), dim=1)
                == self.dataset["train_label"]
            ).float()
        )

    def _test_acc(self):
        return torch.mean(
            (
                torch.argmax(self.model(self.dataset["test_input"]), dim=1)
                == self.dataset["test_label"]
            ).float()
        )

def fit_and_validate(
    self, xtrain: np.ndarray, ytrain: np.ndarray, xval: np.ndarray, yval: np.ndarray
) -> Tuple[float, float]:
    """Training and Evaluating the model.

    Parameters
    ----------
    xtrain : np.ndarray of shape (n_instances, 1, n_timepoints)
        The input time series for training.
    ytrain : np.ndarray of shape (n_instances,)
        The labels of the training samples.
    xval : np.ndarray of shape (n_instances, 1, n_timepoints)
        The input time series for validation.
    yval : np.ndarray of shape (n_instances,)
        The labels of the validation samples.

    Returns
    -------
    Tuple[float, float]
        The accuracies on both train and validation sets.
    """
    # Set device
    self.device = "cpu"

    # Prepare datasets
    self.dataset = {
        "train_input": torch.from_numpy(xtrain).float().to(self.device),
        "train_label": torch.from_numpy(ytrain).long().to(self.device),
        "test_input": torch.from_numpy(xval).float().to(self.device),
        "test_label": torch.from_numpy(yval).long().to(self.device),
    }

    # Configure model dimensions
    _, _, self.length_TS = xtrain.shape  # Number of time points
    self.n_classes = len(np.unique(ytrain))

    # Define KAN model layers
    self.width_ = [self.length_TS] + self.width + [self.n_classes]

    # Instantiate KAN model
    self.model = KAN(
        width=self.width_,
        grid=self.grid,
        k=self.k,
        seed=self.random_state,
        device=self.device,
    )
    self.model.to(self.device)

    # Train the model
    self.results = self.model.train(
        self.dataset,
        steps=self.steps,
        metrics=(self._train_acc, self._test_acc),
        loss_fn=torch.nn.CrossEntropyLoss(),
        device=self.device,
    )

    return self.results["_train_acc"][-1], self.results["_test_acc"][-1]

    def get_symbolic_function(self):
        """Generate symbolic functions for the learned model."""
        lib = [
            "x",
            "x^2",
            "x^3",
            "x^4",
            "exp",
            "log",
            "sqrt",
            "tanh",
            "sin",
            "tan",
            "abs",
        ]
        self.model.auto_symbolic(lib=lib)
        self.formulas = self.model.symbolic_formula()[0]

        return [_formula for _formula in self.formulas]


if __name__ == "__main__":
    # Load training and testing datasets
    X_train, y_train = load_from_tsfile("./data/ts_files/train.ts")
    X_test, y_test = load_from_tsfile("./data/ts_files/test.ts")

    # Encode string labels into integers
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # Instantiate classifier
    classifier = KANClassifier(
        width=[50, 30], steps=20, k=3, grid=5, random_state=42
    )

    # Train and evaluate the model
    train_acc, test_acc = classifier.fit_and_validate(
        xtrain=X_train, ytrain=y_train_encoded,
        xval=X_test, yval=y_test_encoded
    )

    print(f"Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

    # Decode the integer labels if needed
    class_labels = label_encoder.inverse_transform(range(len(label_encoder.classes_)))
    print(f"Class Labels: {class_labels}")

checkpoint directory created: ./model
saving model version 0.0


TypeError: Module.train() got an unexpected keyword argument 'steps'

In [34]:
X_train.shape

(6000, 1, 137)

In [32]:
y_train_encoded

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [26]:
from kan import KAN
from kan.utils import create_dataset
import torch
from aeon.datasets._data_loaders import load_from_tsfile

# Set default data type and device
torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

# Create a KAN: 2D inputs, 1D output, and 5 hidden neurons. Cubic spline (k=3), 5 grid intervals (grid=5).
model = KAN(width=[2, 5, 1], grid=3, k=3, seed=42, device=device)

# Load dataset
X_train, y_train = load_from_tsfile("./data/ts_files/train.ts")
X_test, y_test = load_from_tsfile("./data/ts_files/test.ts")

# Map labels to integers
unique_labels = set(y_train)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}

# Prepare the dataset using create_dataset
# The lambda function maps the label values to their integer equivalents
dataset = create_dataset(
    lambda x: torch.tensor([label_mapping[label] for label in y_train], dtype=torch.long),
    n_var=X_train.shape[1],  # Number of variables (features)
    device=device
)

# Pass the dataset to the model
model(dataset)

# Plot the model
model.plot()

Device: cpu
checkpoint directory created: ./model
saving model version 0.0


TypeError: unhashable type: 'slice'

In [19]:
dataset['train_input'].shape

TypeError: tuple indices must be integers or slices, not str